In [24]:
import pandas as pd
import numpy as np
import pyomo.environ as pe
import pyomo.opt as po

# Read in the paramters
params_df = pd.read_csv('DICE2016R3Parameters.csv')
params = dict(zip(params_df.key,params_df.value))

# Convert to numeric
for key in params.keys():
    try:
        params[key] = float(params[key])
    except:
        pass

params["numPeriods"] = int(params["numPeriods"])
params["tstep"] = int(params["tstep"])
params["ifopt"] = int(params["ifopt"])

params_df.head()

,key,value,description
0,numPeriods,100.0,The number of periods
1,fosslim,6000.0,Maximum cumulative extraction fossil fuels (GtC)
2,tstep,5.0,Years per period
3,ifopt,1.0,Indicator where optimized is 1 and base is 0
4,p2018,1.2,Price level 2018 relative to 2010


In [25]:
# Reflation
params["q0"] = params["q0"]*params["p2018"]
params["k0"] = params["k0"]*params["p2018"]
params["cprice0"] = params["cprice0"]*params["p2018"]
params["pback"] = params["pback"]*params["p2018"]
params["a0"] = params["a0"]*params["p2018"]**(1-params["gama"])

In [26]:
# Calculate carbon cycle parameters
params["b11"] = 1 - params["b12"]
params["b21"] = params["b12"] * params["mateq"] / params["mueq"]
params["b22"] = 1 - params["b21"] - params["b23"]
params["b32"] = params["b23"] * params["mueq"] / params["mleq"]
params["b33"] = 1 - params["b32"]

In [27]:
# Calculate static parameters
params["a20"] = params["a2"] # to store the damage parameter for different scenarios
params["sig0"] = params["e0"] /  (params["q0"] * (1 - params["miu0"])) # Carbon intensity in 2010
params["lam"] = params["fco22x"] / params["t2xco2"] # Climate model parameter
params["optlrsav"] = (params["dk"] + 0.004) / (params["dk"] + 0.004 * params["elasmu"] + params["prstp"]) * params["gama"]  # Optimal long-run savings rate used for transversality

In [28]:
# Calculate trajectory of exogenous parameters
def getPopulation(maxPeriods,pop0,popasym,popadj):
    """
    Projected population (l)
    
    pop0 : initial population (in millions)
    popasym : asymptotic population (in millions)
    popadj : growth rate to calibrate to 2050 pop projection
    
    Gams code : loop(t, l(t+1)=l(t)*(popasym/L(t))**popadj ;);
    """
    population = np.zeros(maxPeriods + 1)
    population[1] = pop0

    for t in range(2, maxPeriods + 1):
        population[t] = population[t-1]*(popasym/population[t-1])**popadj

    return population

def getGrowthRateOfProductivity(ga0,dela,num_periods,tstep):
    """
    Growth rate of productivity (ga)
    
    ga0 : initial growth rate for TFP per period
    dela : decline rate of TFP per period
    tstep : duration of a period ( = 5 years)
    
    Gams code : ga(t)=ga0*exp(-dela*5*((t.val-1)));
    """
    productivity_growth = np.zeros(num_periods + 1) 

    for t in range(num_periods):
        productivity_growth[t+1] = ga0 * np.exp(-dela*tstep*t)

    return productivity_growth

def getLevelOfProductivity(a0,ga,num_periods):
    """
    Level of total factor productivity (al)

    a0 : initial level of TFP
    ga : growth rate of productivity

    Gams code : loop(t, al(t+1)=al(t)/((1-ga(t))););
    """
    productivity = np.zeros(num_periods + 1)
    productivity[1] = a0
    
    for t in range(2, num_periods + 1):
        productivity[t] = productivity[t-1]/(1-ga[t-1])
    
    return productivity

def getCumulativeEfficiencyImprovement(gsigma1,dsig,tstep,num_periods):
    """
    Change in sigma (cumulative improvement of energy efficiency)
    gsigma1 : initial growth of sigma (per year)
    dsig : decline rate of decarbonization (per period)
    tstep : duration of a period (= 5 years)
    
    Gams code : loop(t,gsig(t+1)=gsig(t)*((1+dsig)**tstep) ;);
    """
    efficiency = np.zeros(num_periods + 1)
    efficiency[1] = gsigma1
    for t in range(2, num_periods + 1):
        efficiency[t] = efficiency[t-1]*((1+dsig)**tstep)
    
    return efficiency

def getGrowthRate(sig0,gsig,tstep,num_periods):
    """
    CO2-equivalent-emissions output ratio (sigma)
    
    sig0 : Carbon intensity 2010
    gsig : cumulative improvement of energy efficiency (change in sigma)
    tstep : duration of a period (= 5 years)
    
    Gams code :  loop(t,sigma(t+1)=(sigma(t)*exp(gsig(t)*tstep)););
    """
    growth_rate = np.zeros(num_periods + 1)
    growth_rate[1] = sig0
    
    for t in range(2, num_periods + 1):
        growth_rate[t] = growth_rate[t-1]*np.exp(gsig[t-1]*tstep)
        
    return growth_rate

def getBackstopPrice(pback,gback,num_periods):
    """
    Backstop price (pbacktime)
    
    pback : cost of backstop 2010$ per tC02
    gback : initial cost decline backstop cost per period
    
    Gams code : pbacktime(t)=pback*(1-gback)**(t.val-1);
    """
    backstop_price = np.zeros(num_periods + 1)

    for t in range(1, num_periods + 1):
        backstop_price[t] = pback*(1-gback)**(t-1)

    return backstop_price

def getAdjustedCostForBackstop(pbacktime,sigma,expcost2,num_periods):
    """
    Adjusted cost for backstop (cost1)
    
    pbacktime : backstop price
    sigma : GHG output ratio
    expcost2 : exponent of control cost function
    
    Gams code : cost1(t) = pbacktime(t)*sigma(t)/expcost2/1000;
    """
    adj_cost = np.zeros(num_periods + 1)
    
    for t in range(1, num_periods + 1):
        adj_cost[t] = pbacktime[t] * sigma[t] / expcost2 / 1000
    
    return adj_cost

def getEmissionsFromDeforestation(eland0,deland,num_periods):
    """
    Emissions from deforestation (etree)
    
    eland0 : Carbon emissions from land 2015
    deland : decline rate of land emissions per period
    
    Gams code : etree(t) = eland0*(1-deland)**(t.val-1);
    """
    emissions = np.zeros(num_periods + 1)
    
    for t in range(1,num_periods + 1):
        emissions[t] = eland0 * (1 - deland)**(t-1)
    
    return emissions

def getCumulativeEmissionsFromLand(cumtree0, etree,num_periods):
    """
    Cumulative from land (cumetree)
    
    cum0: Initial cumulative emissions from land
    etree : emissions from deforestation
    
    Gams code : loop(t,cumetree(t+1)=cumetree(t)+etree(t)*(5/3.666);)
    """
    cum_tree = np.zeros(num_periods + 1)
    cum_tree[1] = cumtree0
    
    for t in range(2, num_periods + 1) :
        cum_tree[t] = cum_tree[t-1] + etree[t-1]*(5/3.666)
    
    return cum_tree

def getAverageUtilitySocialDiscountRate(prstp,tstep,num_periods):
    """
    Average utility social discount rate (rr)
    
    prstp : Initial rate of social time preference per year
    tstep : duration of a period
    
    Gams code : rr(t) = 1/((1+prstp)**(tstep*(t.val-1)));
    """
    util_disc = np.zeros(num_periods + 1)
    
    for t in range(1,num_periods +1):
        util_disc[t] = 1./((1.+prstp)**(tstep*(t-1)))
    
    return util_disc

def getExogenousForcingOfOtherGreenhouseGases(fex0,fex1,num_periods):
    """
    Exogenous forcing for other greenhouse gases (forcoth)
    
    fex0 : 2015 forcings of non CO2 GHG
    fex1 : 2100 forcings of non CO2 GHG
    
    Gams code : forcoth(t) = fex0 + (1/17)*(fex1-fex0)*(t.val-1)$(t.val lt 18)+ (fex1-fex0)$(t.val ge 18);
    """
    exog_forcing = np.zeros(num_periods + 1)
    
    for t in range(1,num_periods +1):
        if t < 18:
            exog_forcing[t] = fex0 + (1./17.)*(fex1-fex0)*(t-1)
        else:
            exog_forcing[t] = fex1
            
    return exog_forcing

def getCarbonPrice(cprice0,gcprice,tstep,num_periods):
    """
    Base Case Carbon Price (cpricebase)
    
    cprice0 : initial base carbon price
    gcprice : growth rate of base carbon price per year
    tstep : duration of a period
    
    Gams code : cpricebase(t)= cprice0*(1+gcprice)**(5*(t.val-1));
    """
    carbon_price = np.zeros(num_periods + 1)
    
    for t in range(1, num_periods + 1):
        carbon_price[t] = cprice0*(1 + gcprice)**(tstep*(t-1))

    return carbon_price

In [29]:
# Calculate dynamic model parameters
params['l'] = getPopulation(params['numPeriods'],params['pop0'],params['popasym'],params['popadj']) 
params['ga'] = getGrowthRateOfProductivity(params['ga0'],params['dela'],params['numPeriods'],params['tstep'])
params['al'] = getLevelOfProductivity(params['a0'],params['ga'],params['numPeriods'])
params['gsig'] = getCumulativeEfficiencyImprovement(params['gsigma1'],params['dsig'],params['tstep'],params['numPeriods'])
params['sigma'] = getGrowthRate(params['sig0'],params['gsig'],params['tstep'],params['numPeriods'])
params['pbacktime'] = getBackstopPrice(params['pback'],params['gback'],params['numPeriods'])
params['cost1'] = getAdjustedCostForBackstop(params['pbacktime'],params['sigma'],params['expcost2'],params['numPeriods'])
params['etree'] = getEmissionsFromDeforestation(params['eland0'],params['deland'],params['numPeriods'])
params['cumetree'] = getCumulativeEmissionsFromLand(params['cumtree0'], params['etree'], params['numPeriods'])
params['rr'] = getAverageUtilitySocialDiscountRate(params['prstp'],params['tstep'],params['numPeriods'])
params['forcoth'] = getExogenousForcingOfOtherGreenhouseGases(params['fex0'],params['fex1'],params['numPeriods'])
params['cpricebase'] = getCarbonPrice(params['cprice0'],params['gcprice'],params['tstep'],params['numPeriods'])

In [30]:
def diceModel2016(**kwargs):
    
    model = pe.ConcreteModel("Dice2016R3")
    
    ###SETS AND INDICES###
    time_periods = np.arange(1, kwargs['numPeriods']+ 1)
    model.time_periods = pe.Set(initialize=time_periods) # t = 1 .. num_periods
    
    
    # Control rate limit
    def miuBounds(model,t):
        if (t == 1) and (kwargs['ifopt'] == 1):    # if optimal control on emissions
            return (kwargs['miu0'],kwargs['miu0']) # initial value
        elif t < 30:
            return (0.,1.)
        else:
            return (0.,kwargs['limmiu']) # upper limit on control rate after 2150

        
    # Upper and lower bounds for stability    
    def tatmBounds(model,t):
        if t == 1: 
            return (kwargs['tatm0'],kwargs['tatm0'])  # initial value
        else:
            return (0.,12.)  # range for increase temperature in atmosphere

    def toceanBounds(model,t):
        if t == 1: 
            return (kwargs['tocean0'],kwargs['tocean0']) # initial value
        else:
            return (-1.,20.) # range for increase temperature in oceans

    def matBounds(model,t):
        if t == 1: 
            return (kwargs['mat0'],kwargs['mat0']) # initial value
        else:
            return (10.,np.inf) # range for carbon concentration increase in atmosphere

    def muBounds(model,t):
        if t == 1: 
            return (kwargs['mu0'],kwargs['mu0']) # initial value
        else:
            return (100.,np.inf) # range for carbon concentration increase in shallow oceans

    def mlBounds(model,t):
        if t == 1: 
            return (kwargs['ml0'],kwargs['ml0'])  # initial value
        else:
            return (1000.,np.inf) # range for carbon concentration in lower oceans

    def cBounds(model,t):
        return (2.,np.inf) # lower bound for consumption

    def kBounds(model,t):
        if t == 1: 
            return (kwargs['k0'],kwargs['k0']) # initial value
        else:
            return (1.,np.inf) # lower bound for capital stock

    def cpcBounds(model,t):
        return (0.01,np.inf) # lower bound for per capita consumption

    # control variable
    def sBounds(model,t):
        if t <= kwargs['numPeriods'] - 10:
            return (-np.inf,np.inf)
        else:
            return (kwargs['optlrsav'],kwargs['optlrsav']) # constraint saving rate at the end 

    # Ressource limit
    def ccaBounds(model,t):
        if t == 1:
            return (kwargs['cca0'],kwargs['cca0']) # initial value
        else:
            return (-np.inf,kwargs['fosslim']) # maximum cumulative extraction fossil fuels

    ## Declarations    
        
    #Emission control rate GHGs    
    model.MIU = pe.Var(model.time_periods,domain=pe.NonNegativeReals,bounds=miuBounds) 
    
    # Increase in radiative forcing (watts per m2 from 1900)
    model.FORC = pe.Var(model.time_periods,domain=pe.Reals) 
    
    # Increase temperature of atmosphere (degrees C from 1900)
    model.TATM = pe.Var(model.time_periods,domain=pe.NonNegativeReals,bounds=tatmBounds) 
    
    # Increase temperatureof lower oceans (degrees C from 1900)
    model.TOCEAN = pe.Var(model.time_periods,domain=pe.Reals,bounds=toceanBounds) 

    # Carbon concentration increase in atmosphere (GtC from 1750)
    model.MAT = pe.Var(model.time_periods,domain=pe.NonNegativeReals,bounds=matBounds) 

    # Carbon concentration increase in shallow oceans (GtC from 1750)
    model.MU = pe.Var(model.time_periods,domain=pe.NonNegativeReals,bounds=muBounds) 

    # Carbon concentration increase in lower oceans (GtC from 1750)
    model.ML = pe.Var(model.time_periods,domain=pe.NonNegativeReals,bounds=mlBounds) 

    # Total CO2 emissions (GtCO2 per year)
    model.E = pe.Var(model.time_periods,domain=pe.Reals) 
    
    #Industrial emissions (GtCO2 per year)
    model.EIND = pe.Var(model.time_periods,domain=pe.Reals) 

    # Consumption (trillions 2005 US dollars per year)
    model.C = pe.Var(model.time_periods,domain=pe.NonNegativeReals,bounds=cBounds) 

    # Capital stock (trillions 2005 US dollars)
    model.K = pe.Var(model.time_periods,domain=pe.NonNegativeReals,bounds=kBounds) 

    # Per capita consumption (thousands 2005 USD per year)
    model.CPC = pe.Var(model.time_periods,domain=pe.NonNegativeReals,bounds=cpcBounds) 

    # Investment (trillions 2005 USD per year)
    model.I = pe.Var(model.time_periods,domain=pe.NonNegativeReals) 

    # Gross savings rate as fraction of gross world product
    model.S = pe.Var(model.time_periods,domain=pe.Reals,bounds=sBounds) 

    # Real interest rate (per annum)
    model.RI = pe.Var(model.time_periods,domain=pe.Reals) 

    # Gross world product net of abatement and damages (trillions 2005 USD per year)
    model.Y = pe.Var(model.time_periods,domain=pe.NonNegativeReals) 

    # Gross world product GROSS of abatement and damages (trillions 2005 USD per year)
    model.YGROSS = pe.Var(model.time_periods,domain=pe.NonNegativeReals) 

    # Output net of damages equation (trillions 2005 USD per year)
    model.YNET = pe.Var(model.time_periods,domain=pe.Reals) 

    # Damages (trillions 2005 USD per year)
    model.DAMAGES = pe.Var(model.time_periods,domain=pe.Reals) 

    # Damages as fraction of gross output
    model.DAMFRAC = pe.Var(model.time_periods,domain=pe.Reals) 

    # Cost of emissions reductions  (trillions 2005 USD per year)
    model.ABATECOST = pe.Var(model.time_periods,domain=pe.Reals) 

    # Marginal cost of abatement (2005$ per ton CO2)
    model.MCABATE = pe.Var(model.time_periods,domain=pe.Reals) 

    # Cumulative industrial carbon emissions (GTC)
    model.CCA = pe.Var(model.time_periods,domain=pe.Reals,bounds=ccaBounds) 
    
    # Total carbon emissions (GtC)
    model.CCATOT = pe.Var(model.time_periods,domain=pe.Reals)

    # One period utility function
    model.PERIODU = pe.Var(model.time_periods,domain=pe.Reals) 

    # Carbon price (2005$ per ton of CO2)
    model.CPRICE = pe.Var(model.time_periods,domain=pe.Reals) 

    # Period utility
    model.CEMUTOTPER = pe.Var(model.time_periods,domain=pe.Reals) 

    # Welfare function
    model.UTILITY = pe.Var(domain=pe.Reals) 

    ### Equations
    
    ## Emissions and damages

    # eeq(t)
    def emissionsEquation(model,t):
        return (model.E[t] == model.EIND[t] + kwargs['etree'][t])
    
    model.emissionsEquation = pe.Constraint(model.time_periods,rule=emissionsEquation)

    # eindeq(t)
    def industrialEmissions(model,t): 
        # EIND(t) =E= sigma(t) * YGROSS(t) * (1-(MIU(t)));
        return (model.EIND[t] == kwargs['sigma'][t] * model.YGROSS[t] * (1 - model.MIU[t]))

    model.industrialEmissions = pe.Constraint(model.time_periods,rule=industrialEmissions)

    # ccacca(t+1)
    def cumCarbonEmissions(model,t):
        if t == 1:
            return pe.Constraint.Skip # to use boundary value instead of  constraint
        else:
            return (model.CCA[t] == model.CCA[t-1] + (model.EIND[t-1] * kwargs['tstep'] / 3.666))

    model.cumCarbonEmissions = pe.Constraint(model.time_periods,rule=cumCarbonEmissions)

    # ccatoteq(t)
    def totCarbonEmissions(model,t):
        return (model.CCATOT[t] == model.CCA[t] + kwargs['cumetree'][t])
    
    model.totCarbonEmissions = pe.Constraint(model.time_periods,rule=totCarbonEmissions)
    
    # force(t)
    def radiativeForcing(model,t): 
        return (model.FORC[t] == kwargs['fco22x'] * (pe.log(model.MAT[t]/588.0)/pe.log(2)) + kwargs['forcoth'][t])

    model.radiativeForcing = pe.Constraint(model.time_periods,rule=radiativeForcing)

    # damfraceq(t)
    # def damageFraction(model,t):
    #     return (model.DAMFRAC[t] == (kwargs['a1'] * model.TATM[t]) + (kwargs['a2'] * model.TATM[t]**kwargs['a3']))

    def damageFraction(model,t):
        g_t = (kwargs['a1'] * model.TATM[t]) + (kwargs['a2'] * model.TATM[t]**2)
        return (model.DAMFRAC[t] == 1 - (1/(1 + g_t)))
    
    model.damageFraction = pe.Constraint(model.time_periods,rule=damageFraction)


    # dameq(t)
    def damagesConst(model,t):
        return (model.DAMAGES[t] == (model.YGROSS[t] * model.DAMFRAC[t]))

    model.damagesConst = pe.Constraint(model.time_periods,rule=damagesConst)

    # abateeq(t)
    def abatementCost(model,t):
        return (model.ABATECOST[t] == model.YGROSS[t] * kwargs['cost1'][t] * (model.MIU[t]**kwargs['expcost2']))

    model.abatementCost = pe.Constraint(model.time_periods,rule=abatementCost)

    # mcabateeq(t)
    def mcAbatement(model,t): 
  
        exp = kwargs['expcost2'] - 1
        return (model.MCABATE[t] == kwargs['pbacktime'][t] * (model.MIU[t])**exp)
          
    model.mcAbatement = pe.Constraint(model.time_periods,rule=mcAbatement)

    # carbpriceeq(t)            
    def carbon_priceEq(model,t): 
        exp = kwargs['expcost2'] - 1
        return (model.CPRICE[t] == kwargs['pbacktime'][t] * (model.MIU[t])**exp)

    model.carbon_priceEq = pe.Constraint(model.time_periods,rule=carbon_priceEq)

    #  dmiueq(t+1)
    def controlRateLimit(model,t):
        if t == 1:
            return pe.Constraint.Skip
        else:
            return (model.MIU[t] <= model.MIU[t-1] + kwargs['dmiulim'])

    model.controlRateLimit = pe.Constraint(model.time_periods,rule=controlRateLimit)

    ## Climate and carbon cycle
                
    # mmat(t+1)
    def atmosphericConcentration(model,t):
        if t == 1:
            return pe.Constraint.Skip
        else: 
            return (model.MAT[t] == model.MAT[t-1]*kwargs['b11'] + model.MU[t-1] * kwargs['b21'] + model.E[t-1] * kwargs['tstep'] / 3.666)

    model.atmosphericConcentration = pe.Constraint(model.time_periods,rule=atmosphericConcentration)

    # mml(t+1)            
    def lowerOceanConcentration(model,t):
        if t == 1:
            return pe.Constraint.Skip
        else:
            return (model.ML[t] == model.ML[t-1]*kwargs['b33'] + model.MU[t-1] * kwargs['b23'] )

    model.lowerOceanConcentration = pe.Constraint(model.time_periods,rule=lowerOceanConcentration)

    # mmu(t+1)
    def upperOceanConcentration(model,t):
        if t == 1:
            return pe.Constraint.Skip
        else:
            return (model.MU[t] == model.MAT[t-1]*kwargs['b12'] + model.MU[t-1]*kwargs['b22'] + model.ML[t-1]*kwargs['b32'])

    model.upperOceanConcentration = pe.Constraint(model.time_periods,rule=upperOceanConcentration)

    # tatmeq(t+1)
    def atmosphericTemperature(model,t):
        if t == 1:
            return pe.Constraint.Skip
        else: 
            return (model.TATM[t] == model.TATM[t-1] + kwargs['c1'] * ((model.FORC[t] - (kwargs['fco22x']/kwargs['t2xco2'])*model.TATM[t-1]) \
                                                                       - (kwargs['c3'] * (model.TATM[t-1] - model.TOCEAN[t-1]))))

    model.atmosphericTemperature = pe.Constraint(model.time_periods,rule=atmosphericTemperature)

    # toceaneq(t+1)            
    def oceanTemperature(model,t):
        if t == 1:
            return pe.Constraint.Skip
        else:
            return (model.TOCEAN[t] == model.TOCEAN[t-1] + kwargs['c4'] * (model.TATM[t-1] - model.TOCEAN[t-1]))

    model.oceanTemperature = pe.Constraint(model.time_periods,rule=oceanTemperature)

    ## Economic variables            
    
    # ygrosseq(t)
    def grossOutput(model,t):
        coef = kwargs['al'][t]*(kwargs['l'][t]/1000)**(1-kwargs['gama'])
        return (model.YGROSS[t] == coef*(model.K[t]**kwargs['gama']))

    model.grossOutput = pe.Constraint(model.time_periods,rule=grossOutput)

    # # yneteq(t)
    # def netOutput(model,t):
    #     return (model.YNET[t] == model.YGROSS[t] * (1-model.DAMFRAC[t]))

     # yneteq(t)
    def netOutput(model,t):
        return (model.YNET[t] == model.YGROSS[t] * (1-(1-(1-model.DAMFRAC[t])/(1-kwargs['capital_share'] * model.DAMFRAC[t]))))

    model.netOutput = pe.Constraint(model.time_periods,rule=netOutput)

    # yy(t)
    def outputNetEqn(model,t):
        return (model.Y[t] == model.YNET[t] - model.ABATECOST[t])

    model.outputNetEqn = pe.Constraint(model.time_periods,rule=outputNetEqn)

    # cc(t)
    def consumptionEqn(model,t):
        return (model.C[t] == model.Y[t] - model.I[t])

    model.consumptionEqn = pe.Constraint(model.time_periods,rule=consumptionEqn)

    # cpce(t)
    def perCapitaConsumption(model,t):
        return (model.CPC[t] == 1000 * model.C[t] / kwargs['l'][t])
                
    model.perCapitaConsumption = pe.Constraint(model.time_periods,rule=perCapitaConsumption)

    # seq(t)
    def savingsRate(model,t):
        return (model.I[t] ==  model.S[t] * model.Y[t])
                
    model.savingsRate = pe.Constraint(model.time_periods,rule=savingsRate)

    # # kk(t+1)
    # def capitalBalance(model,t):
    #     if t == 1:
    #         return pe.Constraint.Skip # initial value defined by the boundary
    #     else: 
    #         return (model.K[t] == (1 - kwargs['dk'])**kwargs['tstep'] * model.K[t-1] + kwargs['tstep'] * model.I[t-1])

     # kk(t+1)
    def capitalBalance(model,t):
        if t == 1:
            return pe.Constraint.Skip # initial value defined by the boundary
        else: 
            #K(t+1)   =L= (1-dk)**tstep * K(t) + tstep * I(t);
            return (model.K[t] == (1 - kwargs['dk'])**kwargs['tstep'] * model.K[t-1] * (1 - kwargs['capital_share'] * model.DAMFRAC[t-1])**kwargs['tstep'] + kwargs['tstep'] * model.I[t-1])

    model.capitalBalance = pe.Constraint(model.time_periods,rule=capitalBalance)

    # rieq(t)             
    def interestRateEqn(model,t):
        if t == 1:
            return pe.Constraint.Skip # initial value defined by the boundary
        else:
            return (model.RI[t] == (1 + kwargs['prstp']) * (model.CPC[t]/model.CPC[t-1])** (kwargs['elasmu']/kwargs['tstep']) - 1)

    model.interestRateEqn = pe.Constraint(model.time_periods,rule=interestRateEqn)

    # cemutotpereq(t)
    def periodUtilityEqn(model,t):
        return (model.CEMUTOTPER[t] ==  model.PERIODU[t] * kwargs['l'][t] * kwargs['rr'][t])

    model.periodUtilityEqn = pe.Constraint(model.time_periods,rule=periodUtilityEqn)

    # periodueq(t)
    def instUtilityEqn(model,t):
        return (model.PERIODU[t] ==  (((model.C[t] * 1000 / kwargs['l'][t])**(1-kwargs['elasmu'])-1) / (1-kwargs['elasmu'])) - 1)
    
    model.instUtilityEqn = pe.Constraint(model.time_periods,rule=instUtilityEqn)

    # Calculate utility 
    def utilityCalc(model):
        return (model.UTILITY == kwargs['tstep'] * kwargs['scale1'] * pe.summation(model.CEMUTOTPER) + kwargs['scale2'])

    model.utilityCalc = pe.Constraint(rule=utilityCalc)

    # Objective function
    def obj_rule(model):
        return  model.UTILITY
                
    model.OBJ = pe.Objective(rule=obj_rule, sense=pe.maximize)
    model.dual = pe.Suffix(direction=pe.Suffix.IMPORT)

    return model

In [31]:
# Store simulation results in df
def createColumn(variableName, variable, df):
    temp = variable.extract_values()
    df[variableName] = 0.0
    for key,value in temp.items() :
        df[variableName][key] = value

In [32]:
# Solve the DICE model with optional optimization of emissions controls
def solveModel(params,df_results,solver,ifopt=0) :
    
    params["ifopt"] = ifopt
    
    if (ifopt == 0) : # baseline case
        params["a2"] = 0.0 # baseline run : no damage due to GHG
        model_base = diceModel2016(**params)
        results_base = solver.solve(model_base) # solve the model without damage function
    
        # Sanity check
        if (results_base.solver.status != po.SolverStatus.ok):
            print('Check solver not ok ?')
        if (results_base.solver.termination_condition != po.TerminationCondition.optimal):  
            print('Check solver optimality ?')
    
        #  Value of the utility function for the solution
        print('Optimal solution value for intermediate computation :', model_base.OBJ())

        # Hotelling price
        createColumn('PHOTEL',model_base.CPRICE,df_results)
    
        params["a2"] = params["a20"] # restore the damage parameter
        
        # define upper bound on carbon price based on prelim computation
        def cpriceBounds(model,t):
            if t <= params['tnopol'] : # Period before which no emissions controls base
                return (-np.inf,max(df_results["PHOTEL"][t],params["cpricebase"]))
            else:
                return (-np.inf,np.inf) 
    else :
        # No bounds on carbon price
        def cpriceBounds(model,t):
            return (-np.inf,np.inf) 

    
    dicemodel = diceModel2016(**params)
    dicemodel.CPRICE.bounds = cpriceBounds  
    results = solver.solve(dicemodel)
        
    # Sanity check
    if (results.solver.status != po.SolverStatus.ok):
        print('Check solver not ok ?')
    if (results.solver.termination_condition != po.TerminationCondition.optimal):  
        print('Check solver optimality ?')

    #  Value of the utility function for the solution
    print('Optimal solution value for utility :', dicemodel.OBJ())
        
    # Store the results
    createColumn('MIU',dicemodel.MIU,df_results)
    createColumn('FORC',dicemodel.FORC,df_results)
    createColumn('TATM',dicemodel.TATM,df_results)
    createColumn('TOCEAN',dicemodel.TOCEAN,df_results)
    createColumn('MAT',dicemodel.MAT,df_results)
    createColumn('MU',dicemodel.MU,df_results)
    createColumn('ML',dicemodel.ML,df_results)
    createColumn('E',dicemodel.E,df_results)
    createColumn('EIND',dicemodel.EIND,df_results)
    createColumn('C',dicemodel.C,df_results)
    createColumn('K',dicemodel.K,df_results)
    createColumn('CPC',dicemodel.CPC,df_results)
    createColumn('I',dicemodel.I,df_results)
    createColumn('S',dicemodel.S,df_results)
    createColumn('RI',dicemodel.RI,df_results)
    createColumn('Y',dicemodel.Y,df_results)
    createColumn('YGROSS',dicemodel.YGROSS,df_results)
    createColumn('YNET',dicemodel.YNET,df_results)
    createColumn('DAMAGES',dicemodel.DAMAGES,df_results)
    createColumn('DAMFRAC',dicemodel.DAMFRAC,df_results)
    createColumn('ABATECOST',dicemodel.ABATECOST,df_results)
    createColumn('MCABATE',dicemodel.MCABATE,df_results)
    createColumn('CCA',dicemodel.CCA,df_results)
    createColumn('CCATOT',dicemodel.CCATOT,df_results)
    createColumn('PERIODU',dicemodel.PERIODU,df_results)
    createColumn('CPRICE',dicemodel.CPRICE,df_results)
    createColumn('CEMUTOTPER',dicemodel.CEMUTOTPER,df_results)

    # Calculate additional metrics
    df_results['SCC'] = [-1000 * dicemodel.dual[dicemodel.emissionsEquation[t]] / (0.00001 + dicemodel.dual[dicemodel.consumptionEqn[t]]) for t in dicemodel.time_periods]
    df_results['atfrac'] = [(dicemodel.MAT[t].value - 588) / (dicemodel.CCATOT[t].value + 0.000001) for t in dicemodel.time_periods]
    df_results['atfrac2010'] = [(dicemodel.MAT[t].value - params['mat0']) / (0.00001 + dicemodel.CCATOT[t].value - dicemodel.CCATOT[1].value) for t in dicemodel.time_periods]
    df_results['PPM'] = [dicemodel.MAT[t].value / 2.13 for t in dicemodel.time_periods]

    return results
    

In [33]:
# Convert periods to years
base_year = 2015
years = np.arange(base_year, base_year + params['numPeriods'] * params['tstep'], params['tstep'])
periods = np.arange(1, params['numPeriods'] + 1)

In [ ]:
# Solve optimal case
params['capital_share'] = 0.3
df_results_opt = pd.DataFrame(index = periods)
solver = po.SolverFactory('ipopt')
results_opt= solveModel(params,df_results_opt,solver,ifopt=1)
df_results_opt['year'] = years
df_results_opt.head()